In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas'

In [ ]:

# Functie om data in te laden
def load_data(file_path):
    return pd.read_excel(file_path)

# Laad de data dictionary
dd = load_data('path_to_data_dictionary.xlsx')

# Functie om relevante variabelen te selecteren
def select_variables(dd):
    return dd[(dd['Meenemen (0=nee, 1= ja, 2=onbekend, 3=computatie)'] == 1) & 
              (dd['0=input, 1=output, 2=onbekend'] != 2)]['Variable name'].tolist()

# Selecteer relevante variabelen
variables = select_variables(dd)

In [ ]:

# Laad alle databestanden
data_files = ['path_to_file1.xlsx', 'path_to_file2.xlsx', ...]  # Voeg hier alle bestandspaden toe
data_frames = [load_data(file) for file in data_files]

# Combineer alle dataframes
combined_data = pd.concat(data_frames, axis=1)

# Selecteer alleen de relevante kolommen
data = combined_data[variables]

In [ ]:

# Analyseer missing data
missing_data = data.isnull().sum() / len(data) * 100
print("Percentage missende waarden per variabele:")
print(missing_data.sort_values(ascending=False))

# Verwijder variabelen met te veel missende waarden (bijvoorbeeld meer dan 50%)
threshold = 50
data = data.loc[:, data.isnull().mean() < threshold/100]

In [ ]:

# Splits de data in continue en categorische variabelen
continuous_vars = data.select_dtypes(include=['float64', 'int64']).columns
categorical_vars = data.select_dtypes(include=['object']).columns

# Imputeer missende waarden
cont_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

data[continuous_vars] = cont_imputer.fit_transform(data[continuous_vars])
data[categorical_vars] = cat_imputer.fit_transform(data[categorical_vars])

In [ ]:

# Normaliseer continue variabelen
scaler = StandardScaler()
data[continuous_vars] = scaler.fit_transform(data[continuous_vars])

In [ ]:

# One-hot encoding voor categorische variabelen
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_cats = encoder.fit_transform(data[categorical_vars])
encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names(categorical_vars))

In [ ]:

# Combineer gecodeerde categorische variabelen met continue variabelen
final_data = pd.concat([data[continuous_vars], encoded_df], axis=1)

In [ ]:

# Split de data in features (X) en target (y)
X = final_data
y = data['target_variable']  # Vervang 'target_variable' door de naam van uw doelvariabele

In [ ]:

# Split de data in training en testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Converteer naar PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)